# Customer Product Selection by Numeric Size
Enter the customer id: 212798
Enter the product: blouse
Enter the size of product: 8
    
Enter the customer id: 45337
Enter the product: blazer
Enter the size of product: 12
    
# Customer Product Selection by Alpha Size
Enter the customer id: 45337
Enter the product: sweater
Enter the size of product: M 


Enter the customer id: 362951
Enter the product: sweatshirt
Enter the size of product: 2XL

In [1]:
#import_libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", None)

In [2]:
#Entering input from customer such as user_id,Product and Size of the product
cust_id = int(input('Enter the customer id: '))
product = str(input('Enter the product: '))
size = input('Enter the size of product: ')

Enter the customer id: 362951
Enter the product: sweatshirt
Enter the size of product: 2XL


In [3]:
#importing_customer_history_library
cust_history = pd.read_csv('customer_purchase_history.csv')
cust_history.rename({'Us_size':'New_size'},inplace=True,axis=1)
cust_history.head(5)

,S_num,Product,Feedback,Item_id,Size,User_id,Category,Size_convention,Sub_category,New_size
0,0,romper,fit,2260466,14,420272,Women's Topwear,Num,Dresses,14
1,1,gown,fit,153475,12,273551,Women's Topwear,Num,Dresses,12
2,2,sheath,fit,1063761,4,360448,Women's Topwear,Num,Dresses,4
3,3,dress,fit,126335,8,909926,Women's Topwear,Num,Dresses,8
4,4,gown,fit,616682,12,151944,Women's Topwear,Num,Dresses,12


In [4]:
#importing_product_master
Product_master = pd.read_csv('Product_Master.csv')
Product_master.rename({'Size_in_alphabet':'New_size','Us_size':'Size'},inplace=True,axis=1)
Product_master.head()

,Sub_category,Brand,New_size,Size,Bust,Waist,Neckline,Arm_length
0,Tops & Blouses,H&M,3XS-XXS,0,30.00,23.50,13.50,23.25
1,Tops & Blouses,H&M,3XS-XXS,1,30.75,24.25,13.75,23.38
2,Tops & Blouses,H&M,XS,2,31.50,25.00,14.00,23.50
3,Tops & Blouses,H&M,XS,3,32.25,25.63,14.00,23.50
4,Tops & Blouses,H&M,S,4,33.00,26.25,14.00,23.50


In [5]:
#importing_product_size_mapping
Product_Size_Mapping = pd.read_csv('Product to size category mapping.csv')
Product_Size_Mapping.head(2)

,Product,Category,Sub_category
0,romper,Women's Topwear,Dresses
1,gown,Women's Topwear,Dresses


In [6]:
#Identifying_Size_Convention
alp=str(cust_history[cust_history['Product']==product]['Size_convention'].iloc[0])
customer_fit_count = cust_history[(cust_history['User_id']==cust_id) & (cust_history['Feedback']=='fit') & (cust_history['Size_convention']==alp)]
customer_fit_count

,S_num,Product,Feedback,Item_id,Size,User_id,Category,Size_convention,Sub_category,New_size
10997,11188,sweatshirt,fit,2183771,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
15074,15346,sweater,fit,2117864,20,362951,Women's Topwear,Alpha,Tops & Blouses,2XL
38465,39137,sweater,fit,2152395,20,362951,Women's Topwear,Alpha,Tops & Blouses,2XL
65708,66848,sweater,fit,2352435,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
71618,72888,sweater,fit,2382861,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
106126,108012,sweater,fit,2455273,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
112898,114889,sweater,fit,2703233,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
130548,132811,cardigan,fit,2094264,14,362951,Women's Topwear,Alpha,Tops & Blouses,L
147271,149805,vest,fit,2138228,14,362951,Women's Topwear,Alpha,Tops & Blouses,L


In [7]:
#CONVERTING BUST SIZE OF EVERY RECORD TO ITS RESPECTIVE MEAN AND GIVE ITS NEAREST VALUE FROM THE RECORD.
#ONLY WORK IN ALPHA CATEGORY PRODUCT 

Alpha=(customer_fit_count['Size_convention'].unique()[0]=='Alpha')
if Alpha:
    list_of_alpha = list(Product_master['New_size'].unique())
    list_of_subcategory = list(Product_master['Sub_category'].unique())
    for alpha in list_of_alpha:
        temp=0
        min_value=0
        for sub in list_of_subcategory:
            alpha_records = Product_master[(Product_master['New_size']==alpha) & (Product_master['Sub_category']==sub)]
            bust_values = list(alpha_records['Bust'])
            avg_bust= np.mean(bust_values)
            temp=abs(avg_bust-bust_values[0])
            min_value=bust_values[0]
            for bust in bust_values:
                min_diff= abs(bust-avg_bust)
                if min_diff<temp:
                    min_value=bust
                    temp=min_diff
            list_of_index=list(Product_master[(Product_master['New_size']==alpha) & (Product_master['Sub_category']==sub)].index)
            for list_index in list_of_index:
                column_index=list(Product_master.columns).index('Bust')
                Product_master.iat[list_index,column_index]=min_value 

In [8]:
#How to convert dtype('O') to dtype('int64') in python
if Alpha:
    print('in Num')
else:
    customer_fit_count['New_size'] = customer_fit_count['New_size'].astype(str).astype('int64')

in Num


In [9]:
#Mergeing both Product_master and customer_history with Alpha  and Numeric
length = len(customer_fit_count['Size_convention'].unique())
if (length>0) and Alpha: #(customer_hist_Dresses_user['Old_size'].unique()[0]=='Alpha'):
    Historycustomer_product = pd.merge(Product_master, customer_fit_count, on=['Sub_category','New_size','Size'])
elif (length>0):
    customer_fit_count.drop('New_size',axis=1,inplace=True)
    Historycustomer_product = pd.merge(Product_master, customer_fit_count, on=['Sub_category','Size'])

In [10]:
#Getting Bust Size Count and respective sizes
if Alpha:
    Fit_bustsize_Count = Historycustomer_product.groupby(['Bust','New_size']).Bust.agg('count').to_frame('Count').sort_values(['Count'],ascending=False).reset_index()
    Fit_bustsize_Count.head()
else:
    Fit_bustsize_Count = Historycustomer_product.groupby(['Bust','Size']).Bust.agg('count').to_frame('Count').sort_values(['Count'],ascending=False).reset_index()
    Fit_bustsize_Count.rename({'Size':'New_size'},inplace=True,axis=1)
    Fit_bustsize_Count.head()

In [11]:
#Calculaing_Confidence
item_count = Fit_bustsize_Count['Count'].sum()
Fit_bustsize_Count['Confidence'] = (Fit_bustsize_Count['Count'] / item_count) * 100
#Fit_bustsize_Count = Fit_bustsize_Count.drop(['conf', 'confidence'], axis =1)
Fit_bustsize_Count

,Bust,New_size,Count,Confidence
0,41.00,L,7,77.777778
1,49.25,2XL,2,22.222222


In [12]:
#Bust_Size to be displayed(But not displayed)
Fit_bustsize_Value = Fit_bustsize_Count.iloc[0:1,]
Fit_bustsize_Value_from_history = Fit_bustsize_Value.iloc[0,0]
Fit_bustsize_Value_from_history

41.0

In [13]:
#Size to be displayed at the end. from the history table
Fit_Size_Value_from_history = Fit_bustsize_Count['New_size'].iloc[0]
Fit_Size_Value_from_history

'L'

In [14]:
#Confidence_level of the bust which has highest count
MaxCount_bustsize = Fit_bustsize_Count.iloc[[0]]
Confidence_from_History = MaxCount_bustsize['Confidence'].iloc[0]
Confidence_from_History

77.77777777777779

In [15]:
#Improved_Confidence_level
if Alpha:
    final_df= Fit_bustsize_Count[['New_size','Count', 'Confidence']].sort_values('New_size', axis=0, ascending=True)
    Improved_Bustsize_Confidence = final_df['Confidence'].sum()
    Improved_Bustsize_Confidence
else:  #.loc[0:1, :]
    final_df= Fit_bustsize_Count[['New_size','Count', 'Confidence']].sort_values('New_size', axis=0, ascending=True).head(2)
    Improved_Bustsize_Confidence = final_df['Confidence'].sum()
    Improved_Bustsize_Confidence

In [16]:
Improved_Bustsize_Confidence

100.0

In [17]:
#Displayed the final confidence(For Understanding)
final_df

,New_size,Count,Confidence
1,2XL,2,22.222222
0,L,7,77.777778


In [18]:
# This Part Explains Customer Currently Selected Product
customer_mapping_product = Product_Size_Mapping[(Product_Size_Mapping['Product']==product)]
customer_mapping_product.head(2)

,Product,Category,Sub_category
3855,sweatshirt,Women's Topwear,Tops & Blouses
3935,sweatshirt,Women's Topwear,Tops & Blouses


In [19]:
#Mergeing customer_mapping_product with product_master
Selected_Product_Mapping = pd.merge(customer_mapping_product, Product_master, on =['Sub_category'])
Selected_Product_Mapping.head(2)

,Product,Category,Sub_category,Brand,New_size,Size,Bust,Waist,Neckline,Arm_length
0,sweatshirt,Women's Topwear,Tops & Blouses,H&M,3XS-XXS,0,30.0,23.50,13.50,23.25
1,sweatshirt,Women's Topwear,Tops & Blouses,H&M,3XS-XXS,1,30.0,24.25,13.75,23.38


In [20]:
#GEtting Size Convention for both alpha and numeric
if Alpha:
    print(size)
    Selected_product_mapping = Selected_Product_Mapping[(Selected_Product_Mapping['Product']== product) & (Selected_Product_Mapping['New_size']== size)]
    Selected_product_mapping = Selected_product_mapping.drop_duplicates()   #subset=['Product', 'Size'], keep='first'
    Selected_product_mapping
else:
    Selected_product_mapping = Selected_Product_Mapping[(Selected_Product_Mapping['Product']== product) & (Selected_Product_Mapping['Size']== int(size))]
    Selected_product_mapping = Selected_product_mapping.drop_duplicates()
    Selected_product_mapping

2XL


In [21]:
Selected_product_Size_mapping = Selected_product_mapping['Size'].iloc[0]
Selected_product_Size_mapping

20

In [22]:
#Selected Product Bust_Size
Selected_bustsize = Selected_product_mapping[['Bust']]
Selected_bustsize_by_Customer = Selected_bustsize.iloc[0,0]
Selected_bustsize_by_Customer

49.25

In [23]:
#Selected Product Bust_Size
Selected_bustsize = Selected_product_mapping[['Bust']]
Selected_bustsize_by_Customer = Selected_bustsize.iloc[0,0]
Selected_bustsize_by_Customer

49.25

In [24]:
##Size to be displayed  at the end for the product chosen by the customer
if Alpha:
    Selected_product_Size_ByCustomer = Selected_product_mapping[['New_size']].iloc[0,0]
    Selected_product_Size_ByCustomer
else:
    Selected_product_Size_ByCustomer = Selected_product_mapping[['Size']].iloc[0,0]
    Selected_product_Size_ByCustomer

In [25]:
Selected_product_Size_ByCustomer

'2XL'

In [26]:
#Selected Product Confidence for alpha and numeric
if Alpha:
    Selected_product_Size_Alpha=Fit_bustsize_Count[Fit_bustsize_Count['New_size']== Selected_product_Size_ByCustomer]
    Selected_product_Size_Confidence = Selected_product_Size_Alpha['Confidence'].iloc[0]
    Selected_product_Size_Confidence
else:
    Selected_product_Size=Fit_bustsize_Count[Fit_bustsize_Count['New_size']== Selected_product_Size_mapping]
    Selected_product_Size_Confidence = Selected_product_Size['Confidence'].iloc[0]
    Selected_product_Size_Confidence

In [27]:
#Selected Product Bust size fo checking purpose
selected_prod_Bust_Size = Selected_product_mapping['Bust'].iloc[0]
selected_prod_Bust_Size

49.25

In [28]:
#Comparing BustSize but displaying the US Size

if(Fit_bustsize_Value_from_history == selected_prod_Bust_Size):
        print('The Recommended size is:',Fit_Size_Value_from_history)
        print('The Confidence level of Selected Product is:',Improved_Bustsize_Confidence)
else:
    print('The Selected Size is {}'.format(Selected_product_Size_ByCustomer))
    print('Recommended size is %s, Confidence level %.2f'%(Fit_Size_Value_from_history,Improved_Bustsize_Confidence),end="%")

The Selected Size is 2XL
Recommended size is L, Confidence level 100.00%